In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/dataset/Concrete_Data.csv")
df

,Cement,Slag,FlyAsh,Water,Superplasticizer,CoarseAggregate,FineAggregate,Age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [ ]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=56)


In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [ ]:
X_train.shape

(824, 8)

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense

In [ ]:
input = Input(shape = X_train.shape[1:])
x = Dense(units=100, activation="relu")(input)
x = Dense(units=50, activation="relu")(x)

output = Dense(units=1, activation="linear")(x)

In [ ]:
model = Model(inputs=[input], outputs=[output])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense (Dense)               (None, 100)               900       
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 6001 (23.44 KB)
Trainable params: 6001 (23.44 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from keras import backend as k

def r2_keras(y_test, y_pred):
  SS_res = k.sum(k.square(y_test-y_pred))
  SS_tot = k.sum(k.square(y_test-k.mean(y_test)))
  return (1-SS_res/(SS_tot+k.epsilon()))

In [ ]:
model.compile(loss="mse", optimizer="adam", metrics=[r2_keras])

In [ ]:
model.fit(X_train, y_train, epochs=150, validation_split=0.1, batch_size=64)

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
test_accuracy

7/7 [==============================] - 0s 4ms/step - loss: 42.4013 - r2_keras: 0.8477


0.8476678729057312